In [1]:
##Building a vectordb
from langchain_chroma import Chroma 
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
loader=TextLoader("speech.txt") 
data=loader.load()
data

[Document(metadata={'source': 'speech.txt'}, page_content='"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.\n\nWhen I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."')]

In [3]:
#SPlit
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=0)
splits=text_splitter.split_documents(data)

In [4]:
embeddings=OllamaEmbeddings()
vecotrdb=Chroma.from_documents(documents=splits,embedding=embeddings)
vecotrdb

C:\Users\user\AppData\Local\Temp\ipykernel_18436\533675856.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings()


In [5]:
## query it 
query="At what age did the speaker read the quote about living each day as if it were your last?"
docs=vecotrdb.similarity_search(query)
docs[0].page_content

'When I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."'

In [6]:
## Save 
vectordb=Chroma.from_documents(documents=splits,embedding=embeddings,persist_directory="./chroma_db")

In [7]:
##Load 

db2=Chroma(persist_directory="./chroma_db",embedding_function=embeddings)
docs=db2.similarity_search(query)
docs[0].page_content

'When I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."'

In [8]:
##Similarity search with score 
docs=vectordb.similarity_search_with_score(query)
docs

[(Document(id='eb6b41d8-bf21-4867-acb8-d2c8f43d0b47', metadata={'source': 'speech.txt'}, page_content='When I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."'),
  13838.287109375),
 (Document(id='e3d335d5-98c1-48b5-aeef-5e70ab26b11c', metadata={'source': 'speech.txt'}, page_content='"Your time is limited, so don’t waste it living someone else’s life.\n\nDon’t be trapped by dogma — which is living with the results of other people’s thinking. Don’t let the noise of others’ opinions drown out your own inner voice. And most important, have the courage to follow your heart and intuition.\n\nThey somehow already know what you truly want to become. Everything else is secondary.'),
  21286.21484375)]

In [10]:
## retriever Option
retriever=vectordb.as_retriever() 
retriever.invoke(query)[0].page_content

'When I was 17, I read a quote that went something like:\nIf you live each day as if it was your last, someday you’ll most certainly be right."'